In [1]:
import os 
import tensorflow as tf

2025-01-27 15:45:38.108420: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 15:45:38.171618: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-27 15:45:39.396394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%cd /home/lab/Workspace/e2e-mlflow-dvc

/home/lab/Workspace/e2e-mlflow-dvc


In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/diazmanne/e2e-mlflow-dvc.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="diazmanne"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f4e87115c3d98a2ef151c0469443be9738fa35ff"

In [4]:
model = tf.keras.models.load_model("artifacts/training/trained_model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    test_data: Path
    training_data: Path
    all_params: Dict
    mlflow_uri: str
    params_image_size: List[int]
    params_batch_size: int


# Validation function
def validate_evaluation_config(evaluation_config: EvaluationConfig):
    print(f"Model Path: {evaluation_config.path_of_model}")
    print(f"Test Data Path: {evaluation_config.test_data}")
    print(f"Training Data Path: {evaluation_config.training_data}")
    print(f"MLflow URI: {evaluation_config.mlflow_uri}")
    print(f"Image Size: {evaluation_config.params_image_size}")
    print(f"Batch Size: {evaluation_config.params_batch_size}")
    print(f"All Params: {evaluation_config.all_params}")


In [9]:
evaluation_config = EvaluationConfig(
    path_of_model=Path(evaluation["trained_model_path"]),
    test_data=Path(evaluation["test_data_path"]),
    training_data=Path(evaluation["training_data_path"]),
    all_params=evaluation,
    mlflow_uri=mlflow_uri,
    params_image_size=params_image_size,
    params_batch_size=params_batch_size
)

# Call validation function
validate_evaluation_config(evaluation_config)

NameError: name 'evaluation' is not defined